In [1]:
# Импортируем библиотеки и задаем настройки отображения
import pandas as pd
import pandahouse as ph

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import pingouin as pg
from scipy import stats as st
from scipy.stats import norm

import requests
from urllib.parse import urlencode

sns.set(
    style="whitegrid",
    rc={'figure.figsize': (20, 7)})
pd.set_option('display.precision', 2)

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [2]:
# зададим параметры подключения к ClickHouse
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'********'
                     }

In [3]:
# создаем запрос по задаче 1
q = '''
    SELECT COUNT(st_id) AS diligent_student
    FROM (
        SELECT st_id
        FROM {db}.peas
        WHERE toMonth(timest) = (SELECT MAX(toMonth(timest)) FROM {db}.peas)
        GROUP BY st_id
        HAVING SUM(correct) >= 20)
    
    '''
# отправляем запрос и записываем результат в датафрейм
diligent = ph.read_clickhouse(query=q, connection=connection_default)
print(f' Количество усердных студентов, решивших более 20 заданий за текущий месяц {diligent.diligent_student.min()}')

 Количество усердных студентов, решивших более 20 заданий за текущий месяц 136


In [4]:
# создаем запрос по задаче 2
q = '''
    SELECT test_grp AS group,
        COUNT(DISTINCT studs.st_id) AS users,
        COUNT(DISTINCT CASE WHEN money>0 THEN a2.st_id END) AS payed_users,
        COUNT(DISTINCT CASE WHEN money>0 AND sum_correct>10 THEN a1.st_id END) AS payed_active_users,
        COUNT(DISTINCT CASE WHEN sum_correct>10 THEN a1.st_id END)  AS users_active,
        COUNT(DISTINCT CASE WHEN money>0 AND sum_correct_math>=2 AND a2.subject = 'Math' THEN a2.st_id END) AS payed_active_math,
        COUNT(DISTINCT CASE WHEN sum_correct_math>=2 THEN a1.st_id END) AS users_math,
        SUM(money)/COUNT(DISTINCT studs.st_id) AS ARPU,
        SUM(CASE WHEN sum_correct>10 THEN money END)/COUNT(DISTINCT CASE WHEN sum_correct>10 THEN a1.st_id END) AS ARPAU,
        COUNT(DISTINCT a2.st_id)/COUNT(DISTINCT studs.st_id) AS CR_common,
        COUNT(DISTINCT CASE WHEN sum_correct>10 AND money>0 THEN a2.st_id END)/COUNT(DISTINCT CASE WHEN sum_correct>10 THEN a1.st_id END) AS CR_active,
        COUNT(DISTINCT CASE WHEN money>0 AND sum_correct_math>=2 AND a2.subject = 'Math' THEN a2.st_id END)/COUNT(DISTINCT CASE WHEN sum_correct_math>=2 THEN a1.st_id END) AS CR_math  
FROM
    (SELECT st_id, 
        SUM(correct) AS sum_correct, 
        SUM(CASE WHEN subject == 'Math' THEN correct END) AS sum_correct_math 
        FROM {db}.peas 
        GROUP BY st_id ) a1
    FULL JOIN {db}.studs AS studs ON studs.st_id = a1.st_id
    FULL JOIN  {db}.final_project_check AS a2 ON studs.st_id = a2.st_id
GROUP BY test_grp
   '''

# отправляем запрос и записываем результат в датафрейм
sql = ph.read_clickhouse(query=q, connection=connection_default)

In [5]:
display(sql.rename(columns={'group': 'Группа', 
                            'users': 'Число пользователей',
                            'payed_users': 'Число платящих пользователей',
                            'payed_active_users': 'Число активных платящих пользователей',
                            'users_active': 'Число активных пользователей',
                            'payed_active_math': 'Число активных платящих пользователей по математике',
                            'users_math': 'Число активных пользователей по математике',
                            'ARPU': 'Средний чек, руб',
                            'ARPAU': 'Средний чек активного пользователя, руб',
                            'CR_common': 'Конверсия в покупку',
                            'CR_active': 'Конверсия в покупку активного пользователя',
                            'CR_math': 'Конверсия в покупку активных пользователей по математике'})
        [['Группа', 'Число пользователей', 'Средний чек, руб',
          'Средний чек активного пользователя, руб', 'Конверсия в покупку',
          'Конверсия в покупку активного пользователя', 'Конверсия в покупку активных пользователей по математике']])

,Группа,Число пользователей,"Средний чек, руб","Средний чек активного пользователя, руб",Конверсия в покупку,Конверсия в покупку активного пользователя,Конверсия в покупку активных пользователей по математике
0,control,305,4540.98,10393.70,0.05,0.11,0.06
1,pilot,295,11508.47,29739.58,0.11,0.26,0.10
